In [1]:
!pip install --break-system-packages transformers torch soundfile

In [15]:
!pip install huggingface_hub transformers

In [27]:
import torch, librosa, numpy as np
from transformers import AutoProcessor, AutoModelForImageTextToText

In [ ]:
# hf_access_token

In [ ]:
MODEL_ID = "google/gemma-3n-E4B-it"
TOKEN="hf_"

In [36]:
processor = AutoProcessor.from_pretrained(MODEL_ID, token=TOKEN)
model = AutoModelForImageTextToText.from_pretrained(
    MODEL_ID, token=TOKEN, device_map="auto", torch_dtype="auto"
)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [38]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "audio", "audio": "call2.wav"},  # local path
            {"type": "text", "text": "Transcribe the Tamil speech and return the final output in English only."},
        ],
    }
]

# IMPORTANT: return_dict=True so you get a mapping suitable for generate(**…)
inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)

# move tensors to device
inputs = {k: (v.to(model.device, dtype=model.dtype) if isinstance(v, torch.Tensor) else v) for k, v in inputs.items()}

with torch.inference_mode():
    output_ids = model.generate(**inputs, max_new_tokens=512)

text = processor.batch_decode(output_ids, skip_special_tokens=True)[0]
print("\n--- ENGLISH TRANSCRIPTION ---\n", text.strip())

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got CUDABFloat16Type instead (while checking arguments for embedding)

In [40]:
# 16 kHz mono recommended
wav, sr = librosa.load("call2.wav", sr=16000, mono=True)
wav = np.asarray(wav, dtype=np.float32)

messages = [{
  "role": "user",
  "content": [
    {"type": "audio", "audio": "<audio:0>"},
    {"type": "text", "text": "Transcribe the Tamil speech and return the final output in English only."},
  ],
}]
prompt = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

# NOTE: batch the audio → pass [wav]
inputs = processor(
    text=prompt,
    audio=[wav],             # <— batch dimension
    sampling_rate=16000,
    return_tensors="pt",
    padding=True,
)

inputs = {k: (v.to(model.device) if isinstance(v, torch.Tensor) else v) for k, v in inputs.items()}

with torch.inference_mode():
    output_ids = model.generate(**inputs, max_new_tokens=512)

text = processor.batch_decode(output_ids, skip_special_tokens=True)[0]
print("\n--- ENGLISH TRANSCRIPTION ---\n", text.strip())

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.38 GiB. GPU 0 has a total capacity of 14.74 GiB of which 3.86 GiB is free. Process 11074 has 10.88 GiB memory in use. Of the allocated memory 10.71 GiB is allocated by PyTorch, and 44.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)